In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# from sklearn.model_selection import TimeSeriesSplit, train_test_split
# import xgboost as xgb
# from sklearn.metrics import mean_squared_error, confusion_matrix, ConfusionMatrixDisplay, f1_score, precision_score, recall_score, accuracy_score

from tsfresh import extract_features
from tsfresh import select_features
from tsfresh.utilities.dataframe_functions import impute, roll_time_series
from tsfresh.feature_extraction import EfficientFCParameters

In [2]:
united_water = pd.read_parquet('data/clean/water.parquet')

In [3]:
data_water = united_water.drop(columns=['raw_rain', 'chk_note_rain', 'chk_fail_rain', 'chk_note_ro', 'chk_fail_ro', 'comment_ro', 'source_ro'])
data_water = data_water['2015-01-01 00:00:00':'2015-01-02 23:59:59']
# data_water = data_water['2015-01-01 00:00:00':'2016-12-31 23:59:59']
# data_water = data_water['2000-01-01 00:00:00':'2015-12-31 23:59:59']

In [4]:
data_water_mini = data_water.copy()[['weir_level_cal', 'ra_rain', 'raw_ro']]
data_water_mini = data_water.copy()[['ra_rain', 'raw_ro']]
data_water_mini = data_water_mini.dropna(subset='raw_ro')
data_water_mini['ra_rain'] = data_water_mini['ra_rain'].fillna(0)
data_water_mini = data_water_mini.reset_index()
data_water_mini['id'] = 'a'
# data_water_mini.sample(100)
extracted_features = extract_features(data_water_mini, column_sort = 'datetime', column_id = 'id')

Feature Extraction: 100%|██████████| 2/2 [00:19<00:00,  9.74s/it]


In [ ]:
impute(extracted_features)
features_filtered = select_features(extracted_features)

In [14]:
rolling = roll_time_series(
    data_water_mini, 
    column_id='id',
    column_sort='datetime'
    # min_timeshift=pd.Timedelta(minutes=5),
    # max_timeshift=pd.Timedelta(days=3)
    )

Rolling: 100%|██████████| 20/20 [00:08<00:00,  2.31it/s]


In [19]:
rolling

,datetime,ra_rain,raw_ro,id
0,2015-01-01 00:00:00,0.0,43.0,"(a, 2015-01-01 00:00:00)"
1,2015-01-01 00:00:00,0.0,43.0,"(a, 2015-01-01 00:05:00)"
2,2015-01-01 00:05:00,0.0,43.0,"(a, 2015-01-01 00:05:00)"
3,2015-01-01 00:00:00,0.0,43.0,"(a, 2015-01-01 00:10:00)"
4,2015-01-01 00:05:00,0.0,43.0,"(a, 2015-01-01 00:10:00)"
...,...,...,...,...
166171,2015-01-02 23:35:00,0.0,40.0,"(a, 2015-01-02 23:55:00)"
166172,2015-01-02 23:40:00,0.0,40.0,"(a, 2015-01-02 23:55:00)"
166173,2015-01-02 23:45:00,0.0,40.0,"(a, 2015-01-02 23:55:00)"
166174,2015-01-02 23:50:00,0.0,40.0,"(a, 2015-01-02 23:55:00)"


In [17]:
features_rolled = extract_features(rolling,
                                    column_id="id", column_sort="datetime",
                                    default_fc_parameters=EfficientFCParameters())

Feature Extraction: 100%|██████████| 20/20 [03:45<00:00, 11.25s/it]


In [18]:
features_rolled.head()

ra_rain__variance_larger_than_standard_deviation  \
a 2015-01-01 00:00:00                                               0.0   
  2015-01-01 00:05:00                                               0.0   
  2015-01-01 00:10:00                                               0.0   
  2015-01-01 00:15:00                                               0.0   
  2015-01-01 00:20:00                                               0.0   

                       ra_rain__has_duplicate_max  ra_rain__has_duplicate_min  \
a 2015-01-01 00:00:00                         0.0                         0.0   
  2015-01-01 00:05:00                         1.0                         1.0   
  2015-01-01 00:10:00                         1.0                         1.0   
  2015-01-01 00:15:00                         1.0                         1.0   
  2015-01-01 00:20:00                         1.0                         1.0   

                       ra_rain__has_duplicate  ra_rain__sum_values  \
a 2015-01-01 00:00:00                     0.0                  0.0   
  2015-01-01 00:05:00                     1.0                  0.0   
  2015-01-01 00:10:00                     1.0                  0.0   
  2015-01-01 00:15:00                     1.0                  0.0   
  2015-01-01 00:20:00                     1.0                  0.0   

                       ra_rain__abs_energy  ra_rain__mean_abs_change  \
a 2015-01-01 00:00:00                  0.0                       NaN   
  2015-01-01 00:05:00                  0.0                       0.0   
  2015-01-01 00:10:00                  0.0                       0.0   
  2015-01-01 00:15:00                  0.0                       0.0   
  2015-01-01 00:20:00                  0.0                       0.0   

                       ra_rain__mean_change  \
a 2015-01-01 00:00:00                   NaN   
  2015-01-01 00:05:00                   0.0   
  2015-01-01 00:10:00                   0.0   
  2015-01-01 00:15:00                   0.0   
  2015-01-01 00:20:00                   0.0   

                       ra_rain__mean_second_derivative_central  \
a 2015-01-01 00:00:00                                      NaN   
  2015-01-01 00:05:00                                      NaN   
  2015-01-01 00:10:00                                      0.0   
  2015-01-01 00:15:00                                      0.0   
  2015-01-01 00:20:00                                      0.0   

                       ra_rain__median  ...  raw_ro__fourier_entropy__bins_5  \
a 2015-01-01 00:00:00              0.0  ...                              NaN   
  2015-01-01 00:05:00              0.0  ...                              NaN   
  2015-01-01 00:10:00              0.0  ...                              NaN   
  2015-01-01 00:15:00              0.0  ...                              NaN   
  2015-01-01 00:20:00              0.0  ...                              NaN   

                       raw_ro__fourier_entropy__bins_10  \
a 2015-01-01 00:00:00                               NaN   
  2015-01-01 00:05:00                               NaN   
  2015-01-01 00:10:00                               NaN   
  2015-01-01 00:15:00                               NaN   
  2015-01-01 00:20:00                               NaN   

                       raw_ro__fourier_entropy__bins_100  \
a 2015-01-01 00:00:00                                NaN   
  2015-01-01 00:05:00                                NaN   
  2015-01-01 00:10:00                                NaN   
  2015-01-01 00:15:00                                NaN   
  2015-01-01 00:20:00                                NaN   

                       raw_ro__permutation_entropy__dimension_3__tau_1  \
a 2015-01-01 00:00:00                                              NaN   
  2015-01-01 00:05:00                                              NaN   
  2015-01-01 00:10:00                                             -0.0   
  2015-01-01 00:15:00                                     